### Forking Sequences

In [1]:
from pyuvm import *

class SeqItem(uvm_sequence_item):
    def __init__(self, name):
        super().__init__(name)
        self.numb = self.result = None

    def __str__(self):
        return f"numb: {self.numb}   result: {self.result}"

class SeqDriver(uvm_driver):
    def run_phase(self):
        while True:
            op_item = self.seq_item_port.get_next_item()
            op_item.result = op_item.numb + 1
            self.seq_item_port.item_done()

class IncSeq(uvm_sequence):
    def body(self):
        print("count up")
        for nn in range(3):
            op = SeqItem("op")
            self.start_item(op)
            op.numb = nn
            self.finish_item(op)
            print(op)

class DecSeq(uvm_sequence):
    def body(self):
        print("count down")
        for nn in range(2,-1,-1):
            op = SeqItem("op")
            self.start_item(op)
            op.numb = nn
            self.finish_item(op)
            print(op)

class TopSeq(uvm_sequence):
    def body(self):
        seqr = ConfigDB().get(None, "", "SEQR")
        inc = IncSeq("inc")
        dec = DecSeq("dec")
        inc_thread = self.fork(inc, seqr)
        dec_thread = self.fork(dec, seqr)
        inc_thread.join()
        dec_thread.join()

class SeqTest(uvm_test):
    def build_phase(self):
        self.seqr = uvm_sequencer("seqr", self)
        self.drvr = SeqDriver("drvr", self)
        ConfigDB().set(None, "*", "SEQR", self.seqr)

    def connect_phase(self):
        self.drvr.seq_item_port.connect(self.seqr.seq_item_export)

    def run_phase(self):
        self.raise_objection()
        self.top = TopSeq("top")
        self.top.start()
        self.drop_objection()

uvm_root().run_test("SeqTest")

count upcount down

numb: 0   result: 1numb: 2   result: 3

numb: 1   result: 2
numb: 1   result: 2
numb: 2   result: 3
numb: 0   result: 1
